In [1]:
from dotenv import load_dotenv
from pymongo import MongoClient
import pandas as pd
import os

In [2]:
# 데이터베이스 연결
load_dotenv()
client = MongoClient(os.getenv('DB_ADR'),
          username=os.getenv('DB_USER'),
          password=os.getenv('DB_PASSWORD'),
          authSource=os.getenv('DB_AuthSource'),
          authMechanism=os.getenv('DB_AuthMechanism'))

db = client.get_database(os.getenv('DB_Collection'))

In [28]:
# 컬렉션에서 시간 필터링('11', '12', '18', '19')
def timefilter(raw_cl, timefiltered_cl):
    target_values=['11', '12', '18', '19']
    regex_pattern = f"^.{{11}}({'|'.join(target_values)}).*$"

    tgt_data=list(raw_cl.find({"time_end": {"$regex":regex_pattern}},{
        '_id':False,
        'uuid':True,
        'time_end':True,
        'destination_lat':True,
        'destination_lng':True}))
    print(len(tgt_data))
    timefiltered_cl.insert_many(tgt_data)



In [ ]:
#2020~23년도 필터링 후 데이터 삽입
raw_cls=['raw_od_uuid_2023', 'raw_od_uuid_2022', 'raw_od_uuid_2021', 'raw_od_uuid_2020']
timefilterd_cls=['timefiltered_2023', 'timefiltered_2022', 'timefiltered_2021', 'timefiltered_2020']

for raw, time in zip(raw_cls, timefilterd_cls):
    raw_cl = db.get_collection(raw)
    timefiltered_cl = db.get_collection(time)
    timefilter(raw_cl,timefiltered_cl)

##### 필터링 데이터 개수
||2023|2022|2021|2020|
|---|---|---|---|---|
|개수|5,909,489|7,527,799|8,627,310|8,907,064|